Clean up

In [124]:
from datetime import datetime, timedelta
import time
import pandas as pd
import os

In [118]:
def cleanUp(df):
  #nullwaarden verwijderen
  df.dropna(inplace=True)

  #tijd aanpassen
  #tijdformaat ..:..:..
  tijd_regex = r'^\d{2}:\d{2}:\d{2}$'
  #omzetten naar datetime
  df['date'] = pd.to_datetime(df['dateDiff']).dt.date
  #filter rijen met formaat
  df = df[df['startTime'].str.match(tijd_regex, na=False) & df['rLength'].str.match(tijd_regex, na=False)].copy()
  #afleveringlengte naar seconden omzetten 
  df['Lengte_sec'] = pd.to_timedelta(df['rLength']).dt.total_seconds().astype(int)
  #kijkers naar int
  df['Kijkers'] = df['rateInK'].dropna().astype(str).str.replace('.', '', regex=False).astype(int)

  #uren met 24+
  def time_cor(rij):
    tijdArr = rij['startTime'].split(':')
    if int(tijdArr[0]) >= 24:
      tijdArr[0] = str(int(tijdArr[0]) - 24).zfill(2)
      rij['date'] += timedelta(days=1)
    rij['startTime'] = ':'.join(tijdArr)
    return rij
  
  df = df.apply(time_cor, axis=1)

  #1 kolom voor beide data
  df['FullDate'] = pd.to_datetime(df['date'].astype(str) 
                                  + " " + df['startTime'].astype(str))
  
  #hour en minute voor join later on
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour
  df['minute'] = pd.to_datetime(df['startTime']).dt.minute

  #kolommen verwijderen die niet nodig meer zijn
  df.drop(['startTime', 'rLength', 'rateInK', 'ranking', 'live'], axis=1, inplace=True)

  #de nieuwe dataframe
  df = df[['FullDate', 'date', 'hour','minute', 'channel', 'description', 'Lengte_sec', 'Kijkers']]

  #hernoemen kolommen
  df.rename(columns={'description':'Programma', 'channel':'Kanaal'}, inplace=True)

  return df

In [119]:
Kijkcijfers = pd.read_csv('./data csv/kijkcijfersdataraw.csv')
Kijkcijfers = cleanUp(Kijkcijfers)
Kijkcijfers.sample(10)

C:\Users\krist\AppData\Local\Temp\ipykernel_326208\78328727.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour
C:\Users\krist\AppData\Local\Temp\ipykernel_326208\78328727.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['minute'] = pd.to_datetime(df['startTime']).dt.minute


,FullDate,date,hour,minute,Kanaal,Programma,Lengte_sec,Kijkers
2289,2017-01-24 20:03:29,2017-01-24,20,3,VTM,FAMILIE,1473,800746
15432,2018-11-12 20:41:13,2018-11-12,20,41,EEN,KINDERZIEKENHUIS 24/7,2878,742683
12518,2018-06-20 23:19:37,2018-06-20,23,19,EEN,HET JOURNAAL LAAT,1077,344139
28133,2020-08-10 21:20:14,2020-08-10,21,20,VIER,HET RAD,2374,405049
7159,2017-09-25 20:37:05,2017-09-25,20,37,VIER,CAMPING KAREN & JAMES,2462,459076
35744,2021-08-25 20:00:06,2021-08-25,20,0,Canvas,TERZAKE,1703,161269
3499,2017-03-25 19:32:45,2017-03-25,19,32,VIER,THE SKY IS THE LIMIT,2450,124355
13782,2018-08-21 18:15:39,2018-08-21,18,15,EEN,"SECOND CHANCE SUMMER, TUSCANY",2547,219697
17733,2019-03-07 18:59:54,2019-03-07,18,59,VTM,NIEUWS 19U VTM,3091,635588
33176,2021-04-19 20:42:24,2021-04-19,20,42,VTM,"CAMPING COPPENS, OP WEG NAAR ZWEDEN",3065,595183


Weerdata mergen

In [120]:
weerData = pd.read_csv('./data csv/weerdataraw.csv')
weerData['timestamp'] = pd.to_datetime(weerData['timestamp'])
#naar zelfde formaat als kijkcijfer datum
weerData['datetime'] = weerData['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

#hour voor join later on
weerData['hour'] = pd.to_datetime(weerData['datetime']).dt.hour
weerData['minute'] = pd.to_datetime(weerData['datetime']).dt.minute
weerData['date'] = pd.to_datetime(weerData['datetime']).dt.date

#verwijder kolom
weerData = weerData.drop(columns=['timestamp'])

weerData = weerData[['datetime', 'date' ,'hour', 'minute', 'temperature_2m', 'apparent_temperature', 
                           'rain', 'snowfall', 'weather_code', 'cloud_cover', 
                           'wind_speed_10m', 'sunshine_duration']]

#hernoemen kolommen
weerData.rename(columns={'temperature_2m':'Temperatuur', 'apparent_temperature':'Gevoelstemp', 'wind_speed_10m': 'Windsnelheid', 'rain':'Regen', 'snowfall': 'Sneeuw', 'weather_code':'Weercode', 'cloud_cover':'Bewolking', 'sunshine_duration':'Zonnenschijn'}, inplace=True)

weerData.sample(10)

,datetime,date,hour,minute,Temperatuur,Gevoelstemp,Regen,Sneeuw,Weercode,Bewolking,Windsnelheid,Zonnenschijn
22377,2019-04-21 09:00:00,2019-04-21,9,0,12.1,9.2,0.0,0.0,0.0,0.0,10.3,3600.00
74372,2025-03-26 20:00:00,2025-03-26,20,0,10.0,8.0,0.0,0.0,0.0,7.0,5.5,2193.77
55630,2023-02-04 22:00:00,2023-02-04,22,0,6.7,4.5,0.0,0.0,3.0,100.0,8.6,0.00
39433,2021-04-01 01:00:00,2021-04-01,1,0,9.7,7.9,0.0,0.0,1.0,32.0,6.2,0.00
51797,2022-08-29 05:00:00,2022-08-29,5,0,13.5,12.5,0.0,0.0,3.0,100.0,8.7,0.00
4063,2017-03-19 07:00:00,2017-03-19,7,0,10.9,7.3,0.0,0.0,3.0,100.0,23.7,0.00
55002,2023-01-09 18:00:00,2023-01-09,18,0,4.5,0.8,1.8,0.0,61.0,98.0,14.8,0.00
58275,2023-05-26 03:00:00,2023-05-26,3,0,9.0,6.3,0.0,0.0,0.0,0.0,13.0,0.00
35974,2020-11-07 22:00:00,2020-11-07,22,0,7.2,5.2,0.0,0.0,1.0,24.0,7.7,0.00
43159,2021-09-03 07:00:00,2021-09-03,7,0,12.3,11.4,0.0,0.0,0.0,0.0,9.7,0.00


Mergen weerdata en kijkcijferdata op FullDate

In [121]:
kijkcijfersWeer = pd.merge(Kijkcijfers, weerData, on=['date', 'hour'], how='left')

kijkcijfersWeer = kijkcijfersWeer[['FullDate', 'date', 'hour', 'Kanaal', 'Programma', 'Lengte_sec', 'Kijkers', 'Temperatuur', 'Gevoelstemp', 'Regen', 'Sneeuw', 'Weercode', 'Bewolking', 'Windsnelheid', 'Zonnenschijn']]

#kijkcijfersWeer.sample(10)

kijkcijfersWeer.to_csv('./data csv/blablableble.csv')

debug box

In [122]:
# Bekijk de kolomnamen in Kijkcijfers en weerData
print(Kijkcijfers.columns)
print(weerData.columns)

#nullrows aanwezig?
print(f"null-rows: \n{kijkcijfersWeer.isnull().sum()}")



Index(['FullDate', 'date', 'hour', 'minute', 'Kanaal', 'Programma',
       'Lengte_sec', 'Kijkers'],
      dtype='object')
Index(['datetime', 'date', 'hour', 'minute', 'Temperatuur', 'Gevoelstemp',
       'Regen', 'Sneeuw', 'Weercode', 'Bewolking', 'Windsnelheid',
       'Zonnenschijn'],
      dtype='object')
null-rows: 
FullDate         0
date             0
hour             0
Kanaal           0
Programma        0
Lengte_sec       0
Kijkers          0
Temperatuur     20
Gevoelstemp     20
Regen           20
Sneeuw          20
Weercode        20
Bewolking       20
Windsnelheid    20
Zonnenschijn    20
dtype: int64


In [ ]:
kijkcijfersWeer.dropna(inplace=True)
#csv aanmaken als het niet bestaat
csv_file_path = os.path.join('.','data csv','kijkcijfersWeerRaw.csv')
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
#data naar csv
kijkcijfersWeer.to_csv(csv_file_path, index=False)